In [3]:
# !usr/bin/env python3
# coding: utf-8

from sklearn import datasets
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import collections

In [6]:
index2word = {} # 単語のidから単語を取り出す辞書
word2index = {} # 単語から単語のidを取り出す辞書
counts = collections.Counter()
dataset = [] # 単語のid番号のリスト

# 単語からインデックス、インデックスから単語の辞書を構築する
with open('ptb.train.txt') as f:
    for line in f:
        for word in line.split():
            if word not in word2index:
                ind = len(word2index)
                word2index[word] = ind
                index2word[ind] = word
            counts[word2index[word]] += 1
            dataset.append(word2index[word])
n_vocab = len(word2index)
datasize = len(dataset)

In [23]:
print(dataset[100])
print(index2word[dataset[100]])

78
more


In [26]:
# Negative Sampling で使うノイズ分布からのサンプル生成器
from chainer.utils import walker_alias
cs = [counts[w] for w in range(len(counts))]
power = np.float32(0.75)
p = np.array(cs, power.dtype)
sampler = walker_alias.WalkerAlias(p)

In [25]:
sampler.sample(5)

array([1166,   25, 1656,   31, 2004], dtype=int32)

In [28]:
# chain クラス
class MyW2V(chainer.Chain):
    def __init__(self, n_vocab, n_units):
        super(MyW2V, self).__init__(
        embed = L.EmbedID(n_vocab, n_units),
        )
        
    def __call__(self, xb, yb, tb):
        xc = Variable(np.array(xb, dtype=np.int32))
        yc = Variable(np.array(yb, dtype=np.int32))
        tc = Variable(np.array(tb, dtype=np.int32))
        fv = self.fwd(xc, yc)
        return F.sigmoid_cross_entropy(fv, tc)
    
    def fwd(self, x, y):
        xv = self.embed(x)
        yv = self.embed(y)
        return F.sum(xv * yv, axis=1)

In [29]:
demb = 100 # 分散表現の次元数
model = MyW2V(n_vocab, demb)
optimizer = optimizers.Adam()
optimizer.setup(model)

In [30]:
ws = 3 # window size
ngs = 5 # negative sampling size

def mkbatset(dataset, ids):
    xb, yb, tb = [], [], []
    for pos in ids:
        xid = dataset[pos]
        for i in range(1, ws):
            p = pos - i
            if p >= 0:
                xb.append(xid)
                yid = dataset[p]
                yb.append(yid)
                tb.append(1)
                for nid in sampler.sample(ngs):
                    xb.append(yid)
                    yb.append(nid)
                    tb.append(0)
            p = pos + i
            if p < datasize:
                xb.append(xid)
                yid = dataset[p]
                yb.append(yid)
                tb.append(1)
                for nid in sampler.sample(ngs):
                    xb.append(yid)
                    yb.append(nid)
                    tb.append(0)
    return [xb, yb, tb]

In [32]:
bs = 100
for epoch in range(10):
    print('epoch: {0}'.format(epoch))
    indexes = np.random.permutation(datasize)
    for pos in range(0, datasize, bs):
        print(epoch, pos)
        ids = indexes[pos:(pos+bs) if (pos+bs) < datasize else datasize]
        xb, yb, tb = mkbatset(dataset, ids)
        model.zerograds()
        loss = model(xb, yb, tb)
        loss.backward()
        optimizer.update()

epoch: 0
0 0
0 100
0 200
0 300
0 400
0 500
0 600
0 700
0 800
0 900
0 1000
0 1100
0 1200
0 1300
0 1400
0 1500
0 1600
0 1700
0 1800
0 1900
0 2000
0 2100
0 2200
0 2300
0 2400
0 2500
0 2600
0 2700
0 2800
0 2900
0 3000
0 3100
0 3200
0 3300
0 3400
0 3500
0 3600
0 3700
0 3800
0 3900
0 4000
0 4100
0 4200
0 4300
0 4400
0 4500
0 4600
0 4700
0 4800
0 4900
0 5000
0 5100
0 5200
0 5300
0 5400
0 5500
0 5600
0 5700
0 5800
0 5900
0 6000
0 6100
0 6200
0 6300
0 6400
0 6500
0 6600
0 6700
0 6800
0 6900
0 7000
0 7100
0 7200
0 7300
0 7400
0 7500
0 7600
0 7700
0 7800
0 7900
0 8000
0 8100
0 8200
0 8300
0 8400
0 8500
0 8600
0 8700
0 8800
0 8900
0 9000
0 9100
0 9200
0 9300
0 9400
0 9500
0 9600
0 9700
0 9800
0 9900
0 10000
0 10100
0 10200
0 10300
0 10400
0 10500
0 10600
0 10700
0 10800
0 10900
0 11000
0 11100
0 11200
0 11300
0 11400
0 11500
0 11600
0 11700
0 11800
0 11900
0 12000
0 12100
0 12200
0 12300
0 12400
0 12500
0 12600
0 12700
0 12800
0 12900
0 13000
0 13100
0 13200
0 13300
0 13400
0 13500
0 13600
0 13700

0 103400
0 103500
0 103600
0 103700
0 103800
0 103900
0 104000
0 104100
0 104200
0 104300
0 104400
0 104500
0 104600
0 104700
0 104800
0 104900
0 105000
0 105100
0 105200
0 105300
0 105400
0 105500
0 105600
0 105700
0 105800
0 105900
0 106000
0 106100
0 106200
0 106300
0 106400
0 106500
0 106600
0 106700
0 106800
0 106900
0 107000
0 107100
0 107200
0 107300
0 107400
0 107500
0 107600
0 107700
0 107800
0 107900
0 108000
0 108100
0 108200
0 108300
0 108400
0 108500
0 108600
0 108700
0 108800
0 108900
0 109000
0 109100
0 109200
0 109300
0 109400
0 109500
0 109600
0 109700
0 109800
0 109900
0 110000
0 110100
0 110200
0 110300
0 110400
0 110500
0 110600
0 110700
0 110800
0 110900
0 111000
0 111100
0 111200
0 111300
0 111400
0 111500
0 111600
0 111700
0 111800
0 111900
0 112000
0 112100
0 112200
0 112300
0 112400
0 112500
0 112600
0 112700
0 112800
0 112900
0 113000
0 113100
0 113200
0 113300
0 113400
0 113500
0 113600
0 113700
0 113800
0 113900
0 114000
0 114100
0 114200
0 114300
0 114400
0

0 194500
0 194600
0 194700
0 194800
0 194900
0 195000
0 195100
0 195200
0 195300
0 195400
0 195500
0 195600
0 195700
0 195800
0 195900
0 196000
0 196100
0 196200
0 196300
0 196400
0 196500
0 196600
0 196700
0 196800
0 196900
0 197000
0 197100
0 197200
0 197300
0 197400
0 197500
0 197600
0 197700
0 197800
0 197900
0 198000
0 198100
0 198200
0 198300
0 198400
0 198500
0 198600
0 198700
0 198800
0 198900
0 199000
0 199100
0 199200
0 199300
0 199400
0 199500
0 199600
0 199700
0 199800
0 199900
0 200000
0 200100
0 200200
0 200300
0 200400
0 200500
0 200600
0 200700
0 200800
0 200900
0 201000
0 201100
0 201200
0 201300
0 201400
0 201500
0 201600
0 201700
0 201800
0 201900
0 202000
0 202100
0 202200
0 202300
0 202400
0 202500
0 202600
0 202700
0 202800
0 202900
0 203000
0 203100
0 203200
0 203300
0 203400
0 203500
0 203600
0 203700
0 203800
0 203900
0 204000
0 204100
0 204200
0 204300
0 204400
0 204500
0 204600
0 204700
0 204800
0 204900
0 205000
0 205100
0 205200
0 205300
0 205400
0 205500
0

0 285800
0 285900
0 286000
0 286100
0 286200
0 286300
0 286400
0 286500
0 286600
0 286700
0 286800
0 286900
0 287000
0 287100
0 287200
0 287300
0 287400
0 287500
0 287600
0 287700
0 287800
0 287900
0 288000
0 288100
0 288200
0 288300
0 288400
0 288500
0 288600
0 288700
0 288800
0 288900
0 289000
0 289100
0 289200
0 289300
0 289400
0 289500
0 289600
0 289700
0 289800
0 289900
0 290000
0 290100
0 290200
0 290300
0 290400
0 290500
0 290600
0 290700
0 290800
0 290900
0 291000
0 291100
0 291200
0 291300
0 291400
0 291500
0 291600
0 291700
0 291800
0 291900
0 292000
0 292100
0 292200
0 292300
0 292400
0 292500
0 292600
0 292700
0 292800
0 292900
0 293000
0 293100
0 293200
0 293300
0 293400
0 293500
0 293600
0 293700
0 293800
0 293900
0 294000
0 294100
0 294200
0 294300
0 294400
0 294500
0 294600
0 294700
0 294800
0 294900
0 295000
0 295100
0 295200
0 295300
0 295400
0 295500
0 295600
0 295700
0 295800
0 295900
0 296000
0 296100
0 296200
0 296300
0 296400
0 296500
0 296600
0 296700
0 296800
0

0 377100
0 377200
0 377300
0 377400
0 377500
0 377600
0 377700
0 377800
0 377900
0 378000
0 378100
0 378200
0 378300
0 378400
0 378500
0 378600
0 378700
0 378800
0 378900
0 379000
0 379100
0 379200
0 379300
0 379400
0 379500
0 379600
0 379700
0 379800
0 379900
0 380000
0 380100
0 380200
0 380300
0 380400
0 380500
0 380600
0 380700
0 380800
0 380900
0 381000
0 381100
0 381200
0 381300
0 381400
0 381500
0 381600
0 381700
0 381800
0 381900
0 382000
0 382100
0 382200
0 382300
0 382400
0 382500
0 382600
0 382700
0 382800
0 382900
0 383000
0 383100
0 383200
0 383300
0 383400
0 383500
0 383600
0 383700
0 383800
0 383900
0 384000
0 384100
0 384200
0 384300
0 384400
0 384500
0 384600
0 384700
0 384800
0 384900
0 385000
0 385100
0 385200
0 385300
0 385400
0 385500
0 385600
0 385700
0 385800
0 385900
0 386000
0 386100
0 386200
0 386300
0 386400
0 386500
0 386600
0 386700
0 386800
0 386900
0 387000
0 387100
0 387200
0 387300
0 387400
0 387500
0 387600
0 387700
0 387800
0 387900
0 388000
0 388100
0

...

9 439500
9 439600
9 439700
9 439800
9 439900
9 440000
9 440100
9 440200
9 440300
9 440400
9 440500
9 440600
9 440700
9 440800
9 440900
9 441000
9 441100
9 441200
9 441300
9 441400
9 441500
9 441600
9 441700
9 441800
9 441900
9 442000
9 442100
9 442200
9 442300
9 442400
9 442500
9 442600
9 442700
9 442800
9 442900
9 443000
9 443100
9 443200
9 443300
9 443400
9 443500
9 443600
9 443700
9 443800
9 443900
9 444000
9 444100
9 444200
9 444300
9 444400
9 444500
9 444600
9 444700
9 444800
9 444900
9 445000
9 445100
9 445200
9 445300
9 445400
9 445500
9 445600
9 445700
9 445800
9 445900
9 446000
9 446100
9 446200
9 446300
9 446400
9 446500
9 446600
9 446700
9 446800
9 446900
9 447000
9 447100
9 447200
9 447300
9 447400
9 447500
9 447600
9 447700
9 447800
9 447900
9 448000
9 448100
9 448200
9 448300
9 448400
9 448500
9 448600
9 448700
9 448800
9 448900
9 449000
9 449100
9 449200
9 449300
9 449400
9 449500
9 449600
9 449700
9 449800
9 449900
9 450000
9 450100
9 450200
9 450300
9 450400
9 450500
9

9 530700
9 530800
9 530900
9 531000
9 531100
9 531200
9 531300
9 531400
9 531500
9 531600
9 531700
9 531800
9 531900
9 532000
9 532100
9 532200
9 532300
9 532400
9 532500
9 532600
9 532700
9 532800
9 532900
9 533000
9 533100
9 533200
9 533300
9 533400
9 533500
9 533600
9 533700
9 533800
9 533900
9 534000
9 534100
9 534200
9 534300
9 534400
9 534500
9 534600
9 534700
9 534800
9 534900
9 535000
9 535100
9 535200
9 535300
9 535400
9 535500
9 535600
9 535700
9 535800
9 535900
9 536000
9 536100
9 536200
9 536300
9 536400
9 536500
9 536600
9 536700
9 536800
9 536900
9 537000
9 537100
9 537200
9 537300
9 537400
9 537500
9 537600
9 537700
9 537800
9 537900
9 538000
9 538100
9 538200
9 538300
9 538400
9 538500
9 538600
9 538700
9 538800
9 538900
9 539000
9 539100
9 539200
9 539300
9 539400
9 539500
9 539600
9 539700
9 539800
9 539900
9 540000
9 540100
9 540200
9 540300
9 540400
9 540500
9 540600
9 540700
9 540800
9 540900
9 541000
9 541100
9 541200
9 541300
9 541400
9 541500
9 541600
9 541700
9

9 622200
9 622300
9 622400
9 622500
9 622600
9 622700
9 622800
9 622900
9 623000
9 623100
9 623200
9 623300
9 623400
9 623500
9 623600
9 623700
9 623800
9 623900
9 624000
9 624100
9 624200
9 624300
9 624400
9 624500
9 624600
9 624700
9 624800
9 624900
9 625000
9 625100
9 625200
9 625300
9 625400
9 625500
9 625600
9 625700
9 625800
9 625900
9 626000
9 626100
9 626200
9 626300
9 626400
9 626500
9 626600
9 626700
9 626800
9 626900
9 627000
9 627100
9 627200
9 627300
9 627400
9 627500
9 627600
9 627700
9 627800
9 627900
9 628000
9 628100
9 628200
9 628300
9 628400
9 628500
9 628600
9 628700
9 628800
9 628900
9 629000
9 629100
9 629200
9 629300
9 629400
9 629500
9 629600
9 629700
9 629800
9 629900
9 630000
9 630100
9 630200
9 630300
9 630400
9 630500
9 630600
9 630700
9 630800
9 630900
9 631000
9 631100
9 631200
9 631300
9 631400
9 631500
9 631600
9 631700
9 631800
9 631900
9 632000
9 632100
9 632200
9 632300
9 632400
9 632500
9 632600
9 632700
9 632800
9 632900
9 633000
9 633100
9 633200
9

9 713500
9 713600
9 713700
9 713800
9 713900
9 714000
9 714100
9 714200
9 714300
9 714400
9 714500
9 714600
9 714700
9 714800
9 714900
9 715000
9 715100
9 715200
9 715300
9 715400
9 715500
9 715600
9 715700
9 715800
9 715900
9 716000
9 716100
9 716200
9 716300
9 716400
9 716500
9 716600
9 716700
9 716800
9 716900
9 717000
9 717100
9 717200
9 717300
9 717400
9 717500
9 717600
9 717700
9 717800
9 717900
9 718000
9 718100
9 718200
9 718300
9 718400
9 718500
9 718600
9 718700
9 718800
9 718900
9 719000
9 719100
9 719200
9 719300
9 719400
9 719500
9 719600
9 719700
9 719800
9 719900
9 720000
9 720100
9 720200
9 720300
9 720400
9 720500
9 720600
9 720700
9 720800
9 720900
9 721000
9 721100
9 721200
9 721300
9 721400
9 721500
9 721600
9 721700
9 721800
9 721900
9 722000
9 722100
9 722200
9 722300
9 722400
9 722500
9 722600
9 722700
9 722800
9 722900
9 723000
9 723100
9 723200
9 723300
9 723400
9 723500
9 723600
9 723700
9 723800
9 723900
9 724000
9 724100
9 724200
9 724300
9 724400
9 724500
9

9 804700
9 804800
9 804900
9 805000
9 805100
9 805200
9 805300
9 805400
9 805500
9 805600
9 805700
9 805800
9 805900
9 806000
9 806100
9 806200
9 806300
9 806400
9 806500
9 806600
9 806700
9 806800
9 806900
9 807000
9 807100
9 807200
9 807300
9 807400
9 807500
9 807600
9 807700
9 807800
9 807900
9 808000
9 808100
9 808200
9 808300
9 808400
9 808500
9 808600
9 808700
9 808800
9 808900
9 809000
9 809100
9 809200
9 809300
9 809400
9 809500
9 809600
9 809700
9 809800
9 809900
9 810000
9 810100
9 810200
9 810300
9 810400
9 810500
9 810600
9 810700
9 810800
9 810900
9 811000
9 811100
9 811200
9 811300
9 811400
9 811500
9 811600
9 811700
9 811800
9 811900
9 812000
9 812100
9 812200
9 812300
9 812400
9 812500
9 812600
9 812700
9 812800
9 812900
9 813000
9 813100
9 813200
9 813300
9 813400
9 813500
9 813600
9 813700
9 813800
9 813900
9 814000
9 814100
9 814200
9 814300
9 814400
9 814500
9 814600
9 814700
9 814800
9 814900
9 815000
9 815100
9 815200
9 815300
9 815400
9 815500
9 815600
9 815700
9

In [33]:
with open('myw2v.model', 'w') as f:
    f.write('%d %d\n' %(len(index2word), 100))
    w = model.embed.W.data
    for i in range(w.shape[0]):
        v = ' '.join(['%f' %(v for v in w[i])])
        f.write('%s %s\n' %(index2word[i], v))

TypeError: a float is required